# 13. 텐서플로를 사용한 신경망 훈련

## 13.2 텐서플로 처음 시작하기
#### 텐서플로에서 텐서 만들기

In [1]:
import tensorflow as tf
import numpy as np
np.set_printoptions(precision=3)

In [2]:
a = np.array([1,2,3], dtype=np.int32)
b = [4,5,6]

t_a = tf.convert_to_tensor(a) #리스트나 넘파이 배열에서 텐서 만들기
t_b = tf.convert_to_tensor(b)

print(t_a)
print(t_b)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([4 5 6], shape=(3,), dtype=int32)


In [3]:
t_ones = tf.ones((2,3))
t_ones.shape

TensorShape([2, 3])

In [4]:
t_ones.numpy() #텐서가 참조하는 값 

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [5]:
const_tensor = tf.constant([1.2, 5, np.pi],dtype=tf.float32) #상수 값을 가진 텐서

print(const_tensor)

tf.Tensor([1.2   5.    3.142], shape=(3,), dtype=float32)


#### 텐서의 데이터 타입과 크기 조작

In [6]:
## tf.cast() : 텐서의 데이터 타입을 원하는 타입으로 바꿀 수 있음

t_a_new = tf.cast(t_a, tf.int64)
print(t_a_new.dtype)

<dtype: 'int64'>


In [7]:
## 텐서 전치하기
t = tf.random.uniform(shape=(3,5))
t_tr = tf.transpose(t)
print(t.shape, '-->', t_tr.shape)

## 텐서 크기 바꾸기
t = tf.zeros((30,))
t_reshape = tf.reshape(t, shape=(5,6))
print(t_reshape.shape)

## 불필요한 차원 삭제하기
t = tf.zeros((1,2,1,4,1))
t_sqz =tf.squeeze(t, axis=(2,4))
print(t.shape, '-->', t_sqz.shape)

(3, 5) --> (5, 3)
(5, 6)
(1, 2, 1, 4, 1) --> (1, 2, 4)


#### 텐서에 수학 연산 적용 

In [11]:
tf.random.set_seed(1)
t1 = tf.random.uniform(shape=(5,2),minval=-1.0, maxval=1.0) # 균등 분포
t2 = tf.random.normal(shape=(5,2), mean=0.0, stddev=1.0) # 표준 정규 분포

#원소별 곱셈
t3 = tf.multiply(t1, t2).numpy()

# 각 열 평균
t4 = tf.math.reduce_mean(t1, axis=0)
print(t4)

# 행렬 곱셈
t5 = tf.linalg.matmul(t1, t2, transpose_b = True)
print(t5.numpy())

# t1을 전치한 뒤 t1_t X t2를 계산
t6 = tf.linalg.matmul(t1, t2, transpose_a=True)
print(t6.numpy())

# 텐서의 노름 계산
norm_t1 = tf.norm(t1, ord=2, axis=1).numpy()
print(norm_t1)

tf.Tensor([0.09  0.207], shape=(2,), dtype=float32)
[[-1.144  1.115 -0.87  -0.321  0.856]
 [ 0.248 -0.191  0.25  -0.064 -0.331]
 [-0.478  0.407 -0.436  0.022  0.527]
 [ 0.525 -0.234  0.741 -0.593 -1.194]
 [-0.099  0.26   0.125 -0.462 -0.396]]
[[-1.711  0.302]
 [ 0.371 -1.049]]
[1.046 0.293 0.504 0.96  0.383]


#### split(), stack(), concat() 함수

In [14]:
## 분할 개수 지정하기

tf.random.set_seed(1)
t = tf.random.uniform((6,))
print(t.numpy())

t_splits = tf.split(t, num_or_size_splits=3)
[item.numpy() for item in t_splits]


## 다른 분할 크기 전달하기
tf.random.set_seed(1)
t = tf.random.uniform((5,))
print(t.numpy())
t_splits = tf.split(t, num_or_size_splits=[3,2])
[item.numpy() for item in t_splits]

[0.165 0.901 0.631 0.435 0.292 0.643]
[0.165 0.901 0.631 0.435 0.292]


[array([0.165, 0.901, 0.631], dtype=float32),
 array([0.435, 0.292], dtype=float32)]

In [15]:
# 두 텐서 연결
A = tf.ones((3,))
B = tf.zeros((2,))
C = tf.concat([A,B], axis=0)
print(C.numpy())

# 텐서 쌓기
A = tf.ones((3,))
B = tf.zeros((3,))
S = tf.stack([A,B], axis=1)
print(S.numpy())

[1. 1. 1. 0. 0.]
[[1. 0.]
 [1. 0.]
 [1. 0.]]


## 13.3 텐서플로 데이터셋 API(tf.data)를 사용하여 입력 파이프라인 구축

#### 텐서에서 텐서플로 데이터셋 만들기 

In [16]:
a = [1.2, 3.4, 7.5, 4.1, 5.0, 1.0]

ds = tf.data.Dataset.from_tensor_slices(a)
print(ds)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float32, name=None)>


In [17]:
for item in ds:
  print(item)

tf.Tensor(1.2, shape=(), dtype=float32)
tf.Tensor(3.4, shape=(), dtype=float32)
tf.Tensor(7.5, shape=(), dtype=float32)
tf.Tensor(4.1, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [18]:
# 배치 만들기
ds_batch = ds.batch(3)
for i, elem in enumerate(ds_batch, 1):
  print('batch {}:'.format(i), elem.numpy())

batch 1: [1.2 3.4 7.5]
batch 2: [4.1 5.  1. ]
